<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#MCD43" data-toc-modified-id="MCD43-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>MCD43</a></span></li><li><span><a href="#Processing" data-toc-modified-id="Processing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Processing</a></span><ul class="toc-item"><li><span><a href="#MCD43A3" data-toc-modified-id="MCD43A3-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>MCD43A3</a></span></li><li><span><a href="#Data-prep" data-toc-modified-id="Data-prep-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data prep</a></span><ul class="toc-item"><li><span><a href="#Define-SRS" data-toc-modified-id="Define-SRS-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Define SRS</a></span></li><li><span><a href="#Permute-x-and-y" data-toc-modified-id="Permute-x-and-y-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Permute x and y</a></span></li><li><span><a href="#Flatten-and-transform" data-toc-modified-id="Flatten-and-transform-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Flatten and transform</a></span></li><li><span><a href="#Reshape" data-toc-modified-id="Reshape-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Reshape</a></span></li></ul></li><li><span><a href="#Check-spatial-references" data-toc-modified-id="Check-spatial-references-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Check spatial references</a></span></li><li><span><a href="#MCD43A1" data-toc-modified-id="MCD43A1-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>MCD43A1</a></span><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Data-prep" data-toc-modified-id="Data-prep-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>Data prep</a></span></li><li><span><a href="#SZN" data-toc-modified-id="SZN-2.4.3"><span class="toc-item-num">2.4.3&nbsp;&nbsp;</span>SZN</a></span></li><li><span><a href="#Quality-flags" data-toc-modified-id="Quality-flags-2.4.4"><span class="toc-item-num">2.4.4&nbsp;&nbsp;</span>Quality flags</a></span></li><li><span><a href="#Albedo" data-toc-modified-id="Albedo-2.4.5"><span class="toc-item-num">2.4.5&nbsp;&nbsp;</span>Albedo</a></span></li><li><span><a href="#BSA" data-toc-modified-id="BSA-2.4.6"><span class="toc-item-num">2.4.6&nbsp;&nbsp;</span>BSA</a></span></li></ul></li></ul></li><li><span><a href="#scratch" data-toc-modified-id="scratch-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>scratch</a></span></li></ul></div>

# MCD43 Albedos for Florida USGS 2018 Evapotranspiration Model Runs
This notebook documents the process for calculating the black, white, and blue sky albedos for Shoemaker et al. 2018 (USGS). Take care of imports first:

In [ ]:
%matplotlib inline

## plotting only -->>
import cartopy.feature as feat
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('font', **{
    'family': 'normal', 
    'weight': 'normal', 
    'size': 16})

## processing -->>
from pyproj import Proj, transform
import xarray as xr     
import numpy as np 
import datetime
import sys

## MCD43

The HDFs for MCD43A1 are unwieldy and we only need six of the ~20 layers so I requested subsets from APPEEARS in netCDF format. Visit [this document](data/RequestMCD43A3.md) for instructions on how to requests identical subsets using the JSON request records included in this repo.

I requested both MCD43A1 and MCD43A3 for Florida for 2018 instead of just MCD43A1 for reasons that I'll explain later. Each products has the data for the whole year in a single netCDF:
* MCD43A1: [data\MCD43A1.006_500m_aid0001.nc](data\MCD43A1.006_500m_aid0001.nc)
* MCD43A3: [data\MCD43A3.006_500m_aid0001.nc](data\MCD43A3.006_500m_aid0001.nc)

## Processing 

### MCD43A3

MCD43A3 has black- and white-sky albedo calculated for local solar noon observations only, so it resembles a piece of our result.

Open the dataset for MCD43A3 with `xr.open_dataset`:

In [ ]:
netcdf = "data/MCD43A3.006_500m_aid0001.nc"

ds = xr.open_dataset(netcdf) # open to xr.Dataset struct
ds = ds.rename({"xdim": "x", "ydim": "y"}) # rename dims x,y to stndrd

ds

Notice the data are conveniently organized along three dimensions *time, x, y*.

### Data prep
Make a few more improvements to the structure of the dataset:
* Add two-dimensional arrays (2) of lat and lon values to improve spatial metadata
* Assign some additional metadata about the quality flags information to the *Mandatory_Quality* variables
* etc.

We need latitudes and longitudes to calculate the solar zenith angle, so use `numpy` and `proj` libraries to generate arrays of lat,lon equivalents for the *x, y* coordinates in the dataset. A few steps are required:

#### Define SRS
Define spatial information for the source x,y arrays and the target lon,lat arrays using `pyproj.Proj`

In [ ]:
ds.crs

I'm not aware of any geoscience application where sinusoidal coordinates are reference to an origin other than lat==0, lon==0. So, this piece is unnecessary, but it stays for the sake of provenance.

I only used the lambda `getpar` to avoid having too many columns. The mess below is assembling the attributes from `ds.crs` into the proj4 string to be consumed by `pyproj.Proj`. Use the EPSG code for WGS84:

In [ ]:
getpar = lambda a: str(ds.crs.attrs[a])

inproj = Proj(" ".join([
    "+proj=sinu",
    "+lon_0="+getpar("longitude_of_central_meridian"),
    "+x_0="+getpar("false_easting"),
    "+y_0="+getpar("false_northing"),
    "+a="+getpar("semi_major_axis"),
    "+b="+getpar("semi_minor_axis"),
    "+units="+"meter +no_defs"]))

outproj = Proj(init="epsg:4326")

inproj

#### Permute x and y
After we expand **x** and **y** to two dimensions, we need immediately flatten them so when they're taken together they represent a sequence of latitude longitude pairs that enumerate over all pixels from top-left to bottom-right of the dataset. 

You can do this with `numpy.meshgrid` then `numpy.array.flatten`. Use `numpy.meshgrid` first:

In [ ]:
xx, yy = np.meshgrid(ds.x.data, ds.y.data)

xx

#### Flatten and transform
Flatten and transform the expanded 2d arrays and pass them as arguments 3 and 4 to `pyproj.transform` to get lon,lat pairs:

In [ ]:
lon1d, lat1d = transform(
    inproj, 
    outproj, 
    xx.flatten(), 
    yy.flatten())

print("First five (lon,lat) pairs:"); list(zip(lon1d[:5], lat1d[:5]))

#### Reshape
Reshape the new lon,lat arrays to match the original shape of the 2d x (`xx`) and y (`yy`) arrays. The **latitude** and **longitude** variables in the final dataset will be arranged along the **x** and **y** dimensions (both are 1d), so that every sinusoidal pixel is assigned a geographic reference:

In [ ]:
lon2d, lat2d = lon1d.reshape(xx.shape), lat1d.reshape(yy.shape)

lat2d

Now we can add them to our dataset add two additional coordinate variables by creating an `xr.DataArray` for each and adding them to the dataset. This gets us closer to full compliance with CF recommendations:

In [ ]:
ds.coords["lat"] = xr.DataArray(
    data=lat2d, 
    coords=[ds.y, ds.x], 
    dims=["y", "x"], 
    attrs=dict(
        standard_name="latitude",
        long_name="latitude coordinate",
        units="degrees_north"))

ds.coords["lon"] = xr.DataArray(
    data=lon2d, 
    coords=[ds.y, ds.x], 
    dims=["y", "x"], 
    attrs=dict(
        standard_name="longitude",
        long_name="longitude coordinate",
        units="degrees_east"))

ds

### Check spatial references

It's probably a good idea to do more thorough review to make sure there are no mistakes. You can do a lot of indexing and stats calculation in a single line of code. But break it out into a couple distinct steps:
1. select the **black-sky albedo** variable
2. plot the first timestep with `cartopy`
3. plot the **gridded averages by month**
4. plot a single line for the daily time series over all of Florida

**Select BSA:**

In [ ]:
bsa = ds["Albedo_BSA_nir"]; bsa    # select the BSA

**Plot the first timestep:**

In [ ]:
fig = plt.figure(figsize=(11, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Sinusoidal.MODIS)
bsa.isel(time=0).plot.pcolormesh(x="x", y="y", ax=ax, robust=True,)

# optional, aesthetics -->>
ax.set_yticklabels([]); ax.set_xticklabels([]);
ax.add_feature(feat.OCEAN, zorder=0);
ax.add_feature(feat.LAND, zorder=0, edgecolor='black');
ax.gridlines(color="white", alpha=0.25); #ax.set_global();

**Plot monthly gridded averages:**

In [ ]:
bsamo = bsa.groupby(bsa.time.dt.month).mean("time")
bsamo.plot.pcolormesh(
    x='x', 
    y='y', 
    col='month', 
    col_wrap=3, 
    robust=True, 
    figsize=(16, 18))
ax = plt.gca()
ax.set_yticklabels([]); ax.set_xticklabels([]);
ax.set_ylabel(""); ax.set_xlabel("");

**The daily average of BSA over the state of Florida for 2018:**

In [ ]:
bsa.mean(["x","y"]).plot(x="time", figsize=(14, 6))

### MCD43A1

The MODIS BRDF/Albedo/NBAR product (MCD43) is led by [Professor Crystal Schaaf at UMass Boston](https://www.umb.edu/spectralmass/terra_aqua_modis/modis_brdf_albedo_product_mcd43). The albedo [source code](proc/actual_albedo_tool/actual_albedo_tool.tar.gz) maintained by her lab are accessible at her [ftp site](ftp://rsftp.eeos.umb.edu/data01/Website/actual_albedo_tool.tar.gz). The [readme](proc/actual_albedo_tool/albedo/readme) is the source of most of the `block text` in the notebook.



#### Intro

#### Data prep

In [ ]:
ds = xr.open_dataset("data/MCD43A1.006_500m_aid0001.nc")
ds = ds.rename({
    "Num_Parameters": "param", 
    "xdim": "x", 
    "ydim": "y"})

ds

Use the same coordinates as MCD43A3:

In [ ]:
latatts = dict(
    standard_name="latitude",
    long_name="latitude coordinate",
    units="degrees_north")

ds.coords["lat"] = xr.DataArray(
    data=lat2d, 
    coords=[ds.y, ds.x], 
    dims=["y", "x"], 
    attrs=latatts)

lonatts = dict(
    standard_name="longitude",
    long_name="longitude coordinate",
    units="degrees_east")

ds.coords["lon"] = xr.DataArray(
    data=lon2d, 
    coords=[ds.y, ds.x], 
    dims=["y", "x"], 
    attrs=lonatts)

ds

**Subset for testing:**

In [ ]:
ds_jan = ds.isel(time=slice(0,30))
ds_jan

#### SZN
The only input to black- and white-sky albedos that doesn't come in MCD43A1 is the solar zenith angle (but, you can get it in MCD43A2 if you wish to use local solar noon). 

*needs explanation...*

We have to calculate solar zenith angles for every pixel, for every timestep in the time series (that's a lot, obv). In this next cell, we import `pysolar`, a widely used solar index calculator for Python, and test the `get_altitude` function with the first day in our time series:

In [ ]:
from pysolar.solar import *

# get a datetime object for Jan 1 2018
date = datetime.datetime.strptime("2018-01-01", "%Y-%m-%d")

# set the timezone to UTC
date = date.replace(tzinfo=datetime.timezone.utc)

# get the solar zenith angle
alt  = get_altitude(42.206, -71.382, date)

alt

Now run the same function with the datetime for timestep 1 and make sure it matches the angle given above:

In [ ]:
# get timestep 1 as a datetime object
timestep1 = ds_jan.time[0].data.item()._to_real_datetime()

# set timezone to utc
timestep1 = timestep1.replace(tzinfo=datetime.timezone.utc)

# get solar zenith angle
alt2 = get_altitude(42.206, -71.382, timestep1)

# check if they are equal
alt==alt2

True, success! 

This amounts to 758,280,200 individual calculations (`pixels*timesteps`); it may take 10 minutes or so on a typical workstation:

In [ ]:
def sza_eval(lat, lon, time):
    """Convert CF to Python datetime."""
    ts = time.data.item()._to_real_datetime()
    ts = ts.replace(tzinfo=datetime.timezone.utc)
    return(get_altitude(lat, lon, ts))
    
# evaluate sza over all arrays using list comprehension 
sza_arr = np.dstack([sza_eval(lat1d, lon1d, t) for t in ds.time])
sza_arr.shape

Array dimensions look correct. We need to reshape the large axis to match the shape of the raster like we did with our 1d lat and lon coordinates:

In [ ]:
sza2d = sza_arr.reshape((1555, 1336, 365))
sza2d.shape

Add the solar zenith angle array to our master dataset as a new variable:

In [ ]:
SZA = xr.DataArray(  #ds_jan["SZA"] = 
    data=sza2d, 
    coords=[ds_jan.y, ds_jan.x, ds_jan.time], # note that we reorder coords in
    dims=["time", "y", "x"],                  # dims argument to match others
    attrs=dict(
        units="degree",
        standard_name="solar zenith angle",
        long_name="solar zenith angle"))

SZA

In [ ]:
ds_jan.max()

#### Quality flags
Reference: https://lpdaac.usgs.gov/products/mcd43a1v006/

Fortunately, MCD43 quality flags are as simple as they come (for MODIS):
```
Value 	Description
0 	    Processed; good quality (full BRDF inversions)
1 	    Processed; see other QA (magnitude BRDF inversions)
2 	    Processed; good quality (full BRDF inversions); Band 6 filled; dead or noisy detectors
3 	    Processed; see other QA (magnitude BRDF inversions); Band 6 filled; dead or noisy detectors
255 	Fill Value
```

We may decide to filter the pixels derived via magnitude BRDF inversions, but for now we will keep all data. They are already weighted to amplify the best observation over a 16 day period.

#### Albedo

Now we finally calculate albedos.

The black-sky albedo polynomial is presented in a more practical way in the readme than the equation given in the user guide:

```shell
$ sed -n '31,53p;53q' proc/actual_albedo_tool/albedo/readme

    Calculation of WSA and BSA
    --------------------------

    The program read the BRDF parameters from out1. Black-sky
    albedos are calculated according to the following polynomial
    albedo representation.
    
                          K=iso       k=vol           k=geo
    G_0k(term 1)           1.0      -0.007574       -1.284909   <- BSA coefficients
    G_1k(term SZN^2)       0.0      -0.070987       -0.166314   <- 
    G_2k(term SZN^3)       0.0       0.307588        0.041840   <- 
    WSA                    1.0       0.189184       -1.377622   <- WSA coefficients

    BSA(SZN,BAND)=
        F_iso(BAND)*(G_0iso + G_1iso*SZN^2 + G_2iso*SZN^3) +
        F_vol(BAND)*(G_0vol + G_1vol*SZN^2 + G_2vol*SZN^3) +
        F_geo(BAND)*(G_0geo + G_1geo*SZN^2 + G_2geo*SNZ^3)

    SZN:  solar zenith angle
    BAND: band wavelength
    OD:   optical depth
    AMT:  aerosol model type
   
```

#### BSA

Define some functions that do the math above.

We also write a simple function to vectorize each calculation over our giant array. It should run significantly faster than a for loop. We can only vectorize over coordinates with matching dimensions, so we will have to loop over the timesteps.

```python
def vectorized(a, b):
    func = lambda x, y: np.sqrt(x ** 2 + y ** 2)
    return xr.apply_ufunc(func, a, b)
```

In [ ]:
iso = lambda sza:  1.0      +  0.0*sza**2      + 0.0*sza**3      # Isotropic
vol = lambda sza: -0.007574 + -0.070987*sza**2 + 0.307588*sza**3 # RossThick
geo = lambda sza: -1.284909 + -0.166314*sza**2 + 0.041840*sza**3 # LiSparseR

def iso_vectorized(param1, sza):
    pfunc = lambda p1, sza: p1*iso(sza)
    return(xr.apply_ufunc(pfunc, param1, sza))
    
# vectorize each function
#vol_vectorized = lambda sza: xr.apply_ufunc(vol, sza)
#geo_vectorized = lambda sza: xr.apply_ufunc(geo, sza)

In [ ]:
sza_input = ds_jan["SZA"]
nir_input1 = ds_jan["BRDF_Albedo_Parameters_nir"].sel(param=0)

iso_nir = iso_vectorized(nir_input1, sza_input)

In [ ]:
sza_input.isel(time=0).plot.pcolormesh(x='x', y='y')

In [ ]:
nir_input1.isel(time=0).plot.pcolormesh(x='x', y='y')

`bsa_vectorized` is the heavy-lifter. We use it to apply the functions (iso, vol, geo) over two billion times.

In [ ]:
def bsa_vectorize(band):
    """Vectorize albedo polynomials over two 3d arrays."""
    
    p1 = lambda par,sza: par*iso(par, sza)
    p2 = lambda par,sza: par*vol(par, sza)
    p3 = lambda par,sza: par*geo(par, sza)
    
    func = lambda p,s : poly(p, s)
    
    return(xr.apply_ufunc(func, par, sza))

bsa_vectorized(iso, par1, ds["SZA"])




## scratch
**Collect some notes and bits of code here**

<img src="http://docs.pysolar.org/en/latest/_images/reference_frame.png" alt="alt text" width="500px">    
*source: [http://docs.pysolar.org/en/latest/#location-calculation](http://docs.pysolar.org/en/latest/#location-calculation)*


In [ ]:
import cartopy.crs as ccrs

bsa.plot.pcolormesh(
    x='x', 
    y='y', 
    col='month', 
    col_wrap=3, 
    robust=True, 
    figsize=(16,18),
    #transform=ccrs.Sinusoidal(),
    #subplot_kws={'projection': ccrs.Sinusoidal(-82, 28)}
)

ax.set_yticklabels([])
ax.set_xticklabels([])

This binary computes black, white, blue sky albedos from input BRDF parameters from MCD43A1 (HDF):
* 
[/code/albedo/actual_albedo_hdf.exe](./code/albedo/actual_albedo_hdf.exe)

```shell

$ ./code/albedo/actual_albedo_hdf.exe

Usage: actual_albedo.exe [-par][-od][-szn][-out]

   -par <par_file>      input MCD43A1 (V005 or V006, in HDF format)
   -od  <optical_depth> input optical depth (float, range: 0.0-1.0)
   -szn <solar_zenith>  input solar zenith angle you want to compute
                        (float, range: 0.0-89.0 degrees)
   -out <albedo_file>   output file to save actual albedo (in HDF format)

```

In [ ]:
import cartopy.crs as ccrs

bsamo.plot.pcolormesh(
    'xdim', 
    'ydim', 
    col='month', 
    col_wrap=3, 
    robust=True, 
    figsize=(16,18),
    #transform=ccrs.Sinusoidal(),
    #subplot_kws={'projection': ccrs.Sinusoidal(-82, 28)}
)

ax.set_yticklabels([])
ax.set_xticklabels([])

**Notes from the R implementation in my other repo**

```R
#####################################################################################
# These functions are called by the loop function in main_albedo.R
#
#::METHODS::
# 
#   https://modis.ornl.gov/documentation.html
#   https://www.umb.edu/spectralmass/terra_aqua_modis/modis_brdf_albedo_product_mcd43
#
# Black-sky Albedo = 
#   Parameters_01 + 
#   Parameters_02 * (-0.007574 + (-0.070987 * szn^2) + (0.307588 * szn^3)) + 
#   Parameters_03 * (-1.284909 + (-0.166314 * szn^2) + (0.041840 * szn^3))
#
# White-sky Albedo = 
#   Parameters_01 + 
#   Parameters_02 * (0.189184) + 
#   Parameters_03 * (-1.377622) 
#
# Actual (Blue-sky) Albedo = 
#   White-sky Albedo * f(optical depth, solar zenith angle, aerosol type, band) + 
#   Black-sky Albedo * (1 - f(optical depth, solar zenith angle, aerosol type, band)) 
#
#####################################################################################
# Albedo calculation constants
#####################################################################################

## Black sky constants

# Isotropic constant
g0iso <- 1.0
g1iso <- 0.0
g2iso <- 0.0

# RossThick constant
g0vol <- -0.007574
g1vol <- -0.070987
g2vol <- 0.307588

# LiSparseR constant
g0geo <- -1.284909
g1geo <- -0.166314
g2geo <- 0.041840

## White sky constants

gIso <- 1.0       # Isotropic
gVol <- 0.189184  # RossThick
gGeo <- -1.377622 # LiSparseR

#####################################################################################
# Albedo functions
#####################################################################################

# Convert SZN degrees to radians
Deg2Rad = 3.1415926535/180
SF = 1 # Scale factor (0.001) already applied to values by ncdf4. Set to 1.

# Black-sky albedo formula
BlackSA <- function(p1arr, p2arr, p3arr, szn){
  sznrad = szn*Deg2Rad
  return((p1arr*SF)+(p2arr*SF)*(g0vol+(g1vol*sznrad^2)+(g2vol*sznrad^3))+(p3arr*SF)*(g0geo+(g1geo*sznrad^2)+(g2geo*sznrad^3)))
}

# White-sky albedo formula
WhiteSA <- function(p1arr, p2arr, p3arr){
  return((p1arr*SF)*gIso+(p2arr*SF)*gVol+(p3arr*SF)*+gGeo)
}

# Actual albedo formula
ActualSA <- function(WSA, BSA, LUTVal){
  return((WSA*LUTVal)+(BSA*(1-LUTVal)))
}

#####################################################################################
# Lookup tables
#####################################################################################

LUT <- function(band, szn, sod){
  if(band == 'vis'){
    return(vislut[paste('X', sprintf('%.2f', sod), sep = ''),round(abs(szn), digits = 0)+1])
  }
  if(band == 'nir'){
    return(nirlut[paste('X', sprintf('%.2f', sod), sep = ''),round(abs(szn), digits = 0)+1])
  }
  if(band == 'shortwave'){
    return(swlut[paste('X', sprintf('%.2f', sod), sep = ''),round(abs(szn), digits = 0)+1])
  }
}
```